In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [3]:
fake_dataset = pd.read_csv(r"C:\Users\abhin\OneDrive\Desktop\DATA\ML Scratch\Fake.csv\Fake.csv")
true_dataset = pd.read_csv(r"C:\Users\abhin\OneDrive\Desktop\DATA\ML Scratch\True.csv\True.csv")
fake_dataset.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true_dataset.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_dataset['class'] = 0
true_dataset['class'] = 1

In [5]:
fake_dataset.shape, true_dataset.shape

((23481, 5), (21417, 5))

In [6]:
merge_dataset = pd.concat([fake_dataset, true_dataset], axis = 0)


In [7]:
merge_shuffled = merge_dataset.sample(frac=1)
merge_shuffled.reset_index(inplace = True)
merge_shuffled

,index,title,text,subject,date,class
0,12046,ACTOR WHO BLEW THROUGH $150 Million Fortune Is...,"LOL! If I was Cage s manager, I d suggest he s...",politics,"Dec 28, 2016",0
1,9142,Cleveland sued over speech rights outside Repu...,CLEVELAND (Reuters) - Three groups planning to...,politicsNews,"June 14, 2016",1
2,3503,Prominent Holocaust Attorney Files MASSIVE La...,Donald Trump is NOT going to like this at all....,News,"December 8, 2016",0
3,13875,LEADING N. CAROLINA NEWSPAPER: Girls Need To A...,The Leftist agenda in action blurring the line...,politics,"May 19, 2016",0
4,14374,Merkel gets strong backing from her party afte...,BERLIN (Reuters) - German Chancellor Angela Me...,worldnews,"November 20, 2017",1
...,...,...,...,...,...,...
44893,5255,"Trump budget plan boosts Pentagon, trims State...",WASHINGTON (Reuters) - The White House will se...,politicsNews,"February 27, 2017",1
44894,7945,These Ladies Just Created The Most Adorable S...,People have all kinds of ways of supporting th...,News,"February 21, 2016",0
44895,3994,Donald Trump’s Own Email Scandal Actually Bro...,This is hypocrisy of the highest order and pro...,News,"October 31, 2016",0
44896,21321,OBAMA USES SPEECH ON FOREIGN SOIL To Disparage...,Obama has a knack for going overseas and dispa...,left-news,"Nov 18, 2015",0


In [8]:
merge_shuffled.isnull().sum()

index      0
title      0
text       0
subject    0
date       0
class      0
dtype: int64

In [9]:
X = merge_shuffled.drop(columns= ['class', 'index', 'text', 'subject', 'date'] , axis = 1)
Y =merge_shuffled['class']
print(X)
print(Y)

                                                   title
0      ACTOR WHO BLEW THROUGH $150 Million Fortune Is...
1      Cleveland sued over speech rights outside Repu...
2       Prominent Holocaust Attorney Files MASSIVE La...
3      LEADING N. CAROLINA NEWSPAPER: Girls Need To A...
4      Merkel gets strong backing from her party afte...
...                                                  ...
44893  Trump budget plan boosts Pentagon, trims State...
44894   These Ladies Just Created The Most Adorable S...
44895   Donald Trump’s Own Email Scandal Actually Bro...
44896  OBAMA USES SPEECH ON FOREIGN SOIL To Disparage...
44897     Are You A Conservative, Progressive Or Muslim?

[44898 rows x 1 columns]
0        0
1        1
2        0
3        0
4        1
        ..
44893    1
44894    0
44895    0
44896    0
44897    0
Name: class, Length: 44898, dtype: int64


In [10]:
port_stem= PorterStemmer()


In [11]:
 def stemming(content):
     stemmed_content = re.sub('[^a-zA-Z]',' ', content)
     stemmed_content = stemmed_content.lower()
     stemmed_content = stemmed_content.split()
     stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
     stemmed_content = ' '.join(stemmed_content)
     return stemmed_content

In [12]:
X['title'] = X['title'].apply(stemming)
X

,title
0,actor blew million fortun worri play ronald re...
1,cleveland su speech right outsid republican co...
2,promin holocaust attorney file massiv lawsuit ...
3,lead n carolina newspap girl need attempt over...
4,merkel get strong back parti talk fail
...,...
44893,trump budget plan boost pentagon trim state de...
44894,ladi creat ador song ever hillari video
44895,donald trump email scandal actual broke law jail
44896,obama use speech foreign soil disparag half am...


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 1)>

In [24]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 1)>
  Coords	Values
  (0, 0)	1.0


In [14]:
X['title'].shape


(44898,)

In [20]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X['title'])
X = vectorizer.transform(X['title'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, stratify = Y, random_state = 2)

In [23]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 329434 stored elements and shape (35918, 13206)>
  Coords	Values
  (0, 530)	0.40818916359309493
  (0, 1389)	0.3903712677758681
  (0, 7828)	0.3871759407883073
  (0, 9224)	0.39519289701563387
  (0, 10145)	0.1808181061440551
  (0, 10568)	0.3247786995769254
  (0, 11072)	0.23838507466526151
  (0, 12933)	0.424152578692794
  (1, 3404)	0.2174873602963153
  (1, 5605)	0.2751357547746392
  (1, 6194)	0.31991746200317905
  (1, 6794)	0.241605857364696
  (1, 7006)	0.2099419990906582
  (1, 7917)	0.3025943704519041
  (1, 8507)	0.3286000705378668
  (1, 10133)	0.35267983895914107
  (1, 11516)	0.3442501075580304
  (1, 11610)	0.3605612123723185
  (1, 11726)	0.3012799979028783
  (1, 12010)	0.09485124677025802
  (2, 1442)	0.4073986383259549
  (2, 1914)	0.31693670286092657
  (2, 2797)	0.34087883370669075
  (2, 3856)	0.44469934975933356
  (2, 5030)	0.39941091653332383
  :	:
  (35915, 11072)	0.2836073462376783
  (35915, 11299)	0.43538492496712566
  (

In [24]:
print(y_train)

25503    1
158      0
6900     1
10047    0
16844    0
        ..
16254    0
40393    0
20684    1
44348    0
9319     0
Name: class, Length: 35918, dtype: int64


In [25]:
model = LogisticRegression()

In [26]:
model.fit(X_train, y_train)

LogisticRegression()

In [27]:
#accuracy score on the training data
training_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(training_prediction, y_train)

In [28]:
print("Accuracy score of the training data :  ", training_data_accuracy)

Accuracy score of the training data :   0.9609109638621304


In [43]:
testing_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(testing_prediction, y_test)
print("Accuracy score of the testing data:  ", testing_data_accuracy)

Accuracy score of the testing data:   0.9406458797327394


In [57]:
x_new = X_test[1]
prediction = model.predict(x_new)
if(prediction == 0):
    print("The news is not fake")
else:
    print("The news is fake")

The news is not fake


In [58]:
y_test.iloc[1]


np.int64(1)